# Test 1: Basic
Just test my analysis framework by using SVJ with CKKW-L and without decay data.

## 1. Import Packages

In [1]:
# The Python Standard Library
import os
import sys
import time
import datetime
import glob
import multiprocessing as mp

# The Third-Party Library
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import prettytable
import uproot
import pyjet
import importlib

# My Packages
import myhep.particle_information_v2 as mypInfo_v2
import myhep.analytical_function_v2 as myaFun_v2
import myhep.analysis_v3 as myAnal_v3
# import myhep.particleinfo_v1 as mypiv1
# import myhep.particlefun_v1 as myafv1

# increase figure showing resolution
%config InlineBackend.figure_format = 'retina'

## 2. Import .root File and Load the Data via class

In [2]:
INPUT_FILE = '/youwei_u3/svj_data_master/scheme_1/root/ckkwl_wo.root'

DATA = uproot.open(INPUT_FILE)['Delphes;1']
GP = mypInfo_v2.classGenParticle(DATA)
Jet = mypInfo_v2.classJet(DATA)
Event = mypInfo_v2.classEvent(DATA)

### 2-1. Check the number of events for each branch

In [3]:
if GP.length == Jet.length == Event.length:
    print("There are {} events in the .root file.".format(GP.length))
else:
    print("""\033[7;31m****** There is the problem for the number of event \
    in the .root file. ******\033[0m""")
#     print("\033[7;31m****** There is the problem for the number of event "
#           "in the .root file. ******\033[0m")
    print("\033[7;31m****** Please check your .root file. ******\033[0m")

There are 19373 events in the .root file.


### 2-2. Define physical quantites

In [4]:
# 2-1. Invariant mass M
def M(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    return np.sqrt((e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2)


# 2-2. Transverse mass MT
def MT(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    ET1, ET2 = np.sqrt(m1**2 + pt1**2), np.sqrt(m2**2 + pt2**2)
    return np.sqrt((ET1+ET2)**2 - (px1+px2)**2 - (py1+py2)**2)


# 2-3. Transverse mass mT is invariant under Lorentz boost along the z direction.
def mT(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    return np.sqrt((e1+e2)**2 - (pz1+pz2)**2)


# 2-4. Transverse energy ET
def ET(m1, pt1, eta1, phi1, m2, pt2, eta2, phi2):
    px1, py1, pz1 = pt1*np.cos(phi1), pt1 * \
        np.sin(phi1), np.sqrt(m1**2+pt1**2)*np.sinh(eta1)
    e1 = np.sqrt(m1**2 + px1**2 + py1**2 + pz1**2)
    px2, py2, pz2 = pt2*np.cos(phi2), pt2 * \
        np.sin(phi2), np.sqrt(m2**2+pt2**2)*np.sinh(eta2)
    e2 = np.sqrt(m2**2 + px2**2 + py2**2 + pz2**2)
    m12 = np.sqrt((e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2)
    return np.sqrt(m12**2 + (px1+px2)**2 + (py1+py2)**2)

## 3. Analyze the Dark Quark Pair in the Parton and Truth Levels

In [5]:
df_xdxdx_23_v2 = myaFun_v2.parse_xdxdx_v2(GP, status=23)
df_xdxdx_71_v2 = myaFun_v2.parse_xdxdx_v2(GP, status=71)
print('-'*80)
df_xdxdx_23_v3 = myAnal_v3.analyze_xdxdx(GP, status=23)
df_xdxdx_71_v3 = myAnal_v3.analyze_xdxdx(GP, status=71)

All events only include 2 particles.
All events only include 2 particles.
--------------------------------------------------------------------------------
For status = 23, all events only include 2 particles.
For status = 71, all events only include 2 particles.


In [6]:
# check v3 is consistent with v2 for top 6 variables
arr_23_v2 = np.transpose(df_xdxdx_23_v2.to_numpy())
arr_71_v2 = np.transpose(df_xdxdx_71_v2.to_numpy())
arr_23_v3 = np.transpose(df_xdxdx_23_v3.to_numpy())
arr_71_v3 = np.transpose(df_xdxdx_71_v3.to_numpy())

sum_23, sum_71 = 0, 0
for i in range(6):
    sum_23 += np.sum(arr_23_v3[i] - arr_23_v2[i])
    sum_71 += np.sum(arr_71_v3[i] - arr_71_v2[i])
    # print(sum_23, sum_71)
# print('-'*7)
print(sum_23, sum_71)

0.0 0.0


##### Conclusion: The results of v3 is the same as v2 results.

## 4. Jet Clustering

### 4-1. Select stable final state particle without/with filtering out dark sector

In [7]:
SFSP_v2, SFSP_filterDM_v2 = myaFun_v2.selectStableFinalStateParticle_filterDM(
    GP, FILTER=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213])
print('-'*80)
SFSP_v3, SFSP_filterDM_v3 = myAnal_v3.selectStableFinalStateParticle(
    GP, filter=[51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213])

The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213].
There are 19373 events with stable final state.
There are 19373 events with stable final state and without DM.
--------------------------------------------------------------------------------
The PID of dark matter are [51, -51, 53, -53, 4900211, -4900211, 4900213, -4900213].
19373 events are stable final state.
19373 events are stable final state without DM.


In [8]:
# test len(v3) = len(v2)
if (len(SFSP_v3) == len(SFSP_v2)) and (len(SFSP_filterDM_v3) == len(SFSP_filterDM_v2)):
    print('yes')
else:
    print('no')


# test len(SFSP) = len(SFSP_filterDM)
if len(SFSP_v3) == len(SFSP_filterDM_v3):
    print('yes')
else:
    print('no')


# test v3 = v2
sum_SFSP, sum_SFSP_filterDM = 0, 0
for i in range(len(SFSP_v3)):
    sum_SFSP += np.sum(SFSP_v3[i] - SFSP_v2[i])
    sum_SFSP_filterDM += np.sum(SFSP_filterDM_v3[i] - SFSP_filterDM_v2[i])
print(sum_SFSP, sum_SFSP_filterDM)

yes
yes
0.0 0.0


##### Conclusion: The v3 is the same as the v2.

### 4-2. Let's do the jet clustering!!

In [9]:
R, jetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 0

PseudoJet_v2 = myaFun_v2.jetClustering(SFSP_v2, R=R,
                                      p=jetClusteringAlgorithm,
                                      pTmin=pTmin_pyjet)
PseudoJet_filterDM_v2 = myaFun_v2.jetClustering(SFSP_filterDM_v2, R=R,
                                               p=jetClusteringAlgorithm,
                                               pTmin=pTmin_pyjet)
print('Done old version')
print('-'*80)
PseudoJet_v3 = myAnal_v3.jetClustering(SFSP_v3, R=R,
                                       p=jetClusteringAlgorithm,
                                       pTmin=pTmin_pyjet)
PseudoJet_filterDM_v3 = myAnal_v3.jetClustering(SFSP_filterDM_v3, R=R,
                                                p=jetClusteringAlgorithm,
                                                pTmin=pTmin_pyjet)
print('Done new version')

Done old version
--------------------------------------------------------------------------------
Done new version


In [10]:
# test len(v3) = len(v2)
if (len(PseudoJet_v3) == len(PseudoJet_v2)) and (len(PseudoJet_filterDM_v3) == len(PseudoJet_filterDM_v2)):
    print('yes')
else:
    print('no')


# test len(PseudoJet) = len(PseudoJet_filterDM)
if len(PseudoJet_v3) == len(PseudoJet_filterDM_v3):
    print('yes')
else:
    print('no')

yes
yes


In [11]:
# check the PseudoJet v2 is consistent with v3
sum_pt, sum_eta, sum_phi, sum_m = 0, 0, 0, 0

for i in range(len(PseudoJet_v3)):
    for j, jet in enumerate(PseudoJet_v3[i]):
        sum_pt += jet.pt - PseudoJet_v2[i][j].pt
        sum_eta += jet.eta - PseudoJet_v2[i][j].eta
        sum_phi += jet.phi - PseudoJet_v2[i][j].phi
        sum_m += jet.mass - PseudoJet_v2[i][j].mass
print(sum_pt, sum_eta, sum_phi, sum_m)

if sum_pt == sum_eta == sum_phi == sum_m == 0:
    print("The result of v3 is the same as v2 result.")
else:
    print("Please check your PseudoJet data.")

0.0 0.0 0.0 0.0
The result of v3 is the same as v2 result.


In [12]:
# check the PseudoJet_filterDM v2 is consistent with v3
sum_pt, sum_eta, sum_phi, sum_m = 0, 0, 0, 0

for i in range(len(PseudoJet_filterDM_v3)):
    for j, jet in enumerate(PseudoJet_filterDM_v3[i]):
        sum_pt += jet.pt - PseudoJet_filterDM_v2[i][j].pt
        sum_eta += jet.eta - PseudoJet_filterDM_v2[i][j].eta
        sum_phi += jet.phi - PseudoJet_filterDM_v2[i][j].phi
        sum_m += jet.mass - PseudoJet_filterDM_v2[i][j].mass
print(sum_pt, sum_eta, sum_phi, sum_m)

if sum_pt == sum_eta == sum_phi == sum_m == 0:
    print("The result of v3 is the same as v2 result.")
else:
    print("Please check your PseudoJet data.")

0.0 0.0 0.0 0.0
The result of v3 is the same as v2 result.


##### Conclusion: The v3 is the same as the v2.

### test (delete)

### Test 1: Check truth level data

In [13]:
# just check top 5 particles in first event
GP.dataframelize(0).head()

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
0,-1,21,4,-1,3,3,0.000000,0.000000e+00,999.900024,0.000000
1,1,21,5,-1,3,3,0.000000,0.000000e+00,-999.900024,0.000000
2,21,43,5,-1,7,8,0.000000,1.062366e+01,-3.775406,-2.971525
3,5000001,22,0,1,6,6,1533.499878,0.000000e+00,-999.900024,0.000000
4,-1,42,9,-1,0,0,0.000000,1.991679e-13,999.900024,-2.971312


In [14]:
# pick up 4900101 with status = 23
dfGP0 = GP.dataframelize(0)
dfGP0_DQ_Status = dfGP0[(abs(dfGP0['PID']) == 4900101) & (dfGP0['Status'] == 23)]
dfGP0_DQ_Status

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
55,4900101,23,44,-1,57,58,10.0,507.201447,0.431692,-0.346168
56,-4900101,23,44,-1,59,59,10.0,496.481354,-1.542102,2.787818


In [15]:
# look for 4900101's mother
dfGP0.iloc[[6, 44, 55, 56]]

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
6,5000001,44,3,-1,44,44,1533.499878,10.623662,-5.111654,0.170067
44,5000001,62,6,-1,55,56,1533.499878,11.379427,-5.042953,-0.007871
55,4900101,23,44,-1,57,58,10.000000,507.201447,0.431692,-0.346168
56,-4900101,23,44,-1,59,59,10.000000,496.481354,-1.542102,2.787818


In [16]:
# look for the variation of 4900101
# D1 and D2 of 55, 56
print(dfGP0.iloc[[57, 58, 59]])
print('-'*80)

# D1 and D2 of 57(4900101), 58(4900021), 59(-4900101)
print(dfGP0.iloc[[60, 61, 62, 63, 64]])
print('-'*80)

# D1 and D2 of 60(4900101), 63(-4900101)
print(dfGP0.iloc[[93, 71]])
print('-'*80)

# D1 and D2 of 93(4900101), 71(-4900101)
# 107~116 are dark mesons
print(dfGP0.iloc[[107, 116, 77]])
print('-'*80)

# D1 and D2 of 77(-4900101)
print(dfGP0.iloc[[87, 88]])
print('-'*80)

# D1 and D2 of 87(-4900101)
print(dfGP0.iloc[[106]])

        PID  Status  M1  M2  D1  D2  Mass          PT       Eta       Phi
57  4900101      51  55  -1  60  61  10.0   31.892319 -0.455998 -0.709771
58  4900021      51  55  -1  62  62   0.0  473.790314  0.472955 -0.322166
59 -4900101      52  56  -1  63  64  10.0  492.740875 -1.542103  2.787818
--------------------------------------------------------------------------------
        PID  Status  M1  M2  D1  D2  Mass          PT       Eta       Phi
60  4900101      51  57  -1  93  93  10.0   36.739040 -0.223569 -0.881083
61  4900021      51  57  -1  68  68   0.0   20.493288  0.165171  0.048696
62  4900021      52  58  -1  65  65   0.0  453.068237  0.472955 -0.322166
63 -4900101      51  59  -1  71  71  10.0  454.629242 -1.542552  2.809824
64  4900021      51  59  -1  69  70   0.0   39.222626 -1.509026  2.529673
--------------------------------------------------------------------------------
        PID  Status  M1  M2   D1   D2  Mass          PT       Eta       Phi
93  4900101      71  6

##### Conclusion: I think the `Status = 51` and `52` are not important! Just radiation which means `51` radiate to `4900021` and `52` doesn't radiate.

In [17]:
# pick up all 4900021
dfGP0_DQ_Status_4900021 = dfGP0[(abs(dfGP0['PID']) == 4900021)]
dfGP0_DQ_Status_4900021

,PID,Status,M1,M2,D1,D2,Mass,PT,Eta,Phi
58,4900021,51,55,-1,62,62,0.0,473.790314,0.472955,-0.322166
61,4900021,51,57,-1,68,68,0.0,20.493288,0.165171,0.048696
62,4900021,52,58,-1,65,65,0.0,453.068237,0.472955,-0.322166
64,4900021,51,59,-1,69,70,0.0,39.222626,-1.509026,2.529673
65,4900021,52,62,-1,66,67,0.0,452.769348,0.472955,-0.322166
66,4900021,51,65,-1,74,74,0.0,244.381042,0.470517,-0.356783
67,4900021,51,65,-1,92,92,0.0,211.695419,0.471106,-0.276790
68,4900021,52,61,-1,94,94,0.0,17.335821,0.165171,0.048696
69,4900021,51,64,-1,72,73,0.0,134.818726,-1.499986,2.744825
70,4900021,51,64,-1,75,76,0.0,9.767427,-1.983904,2.638130


##### Conclusion: All `4900021` are hadronize to #107~#116 and no `Status = 1` event.

### Test 2. PseudoJet with $p_T$ > 20 GeV

In [18]:
# pTmin_pyjet = 20
R, jetClusteringAlgorithm, pTmin_pyjet = 0.4, -1, 20

PseudoJet_filterDM_v3_pt20 = myAnal_v3.jetClustering(SFSP_filterDM_v3, R=R,
                                                     p=jetClusteringAlgorithm,
                                                     pTmin=pTmin_pyjet)
print('Done new version with minimal pT = 20')

Done new version with minimal pT = 20


In [19]:
# check number of events of PseudoJet
# check number of events for top 5 events
print(len(PseudoJet_filterDM_v3_pt20), len(PseudoJet_filterDM_v3))
print('-'*80)
for i in range(5):
    print(len(PseudoJet_filterDM_v3_pt20[i]), len(PseudoJet_filterDM_v3[i]))

19373 19373
--------------------------------------------------------------------------------
2 88
3 154
1 163
1 197
4 177


##### Conclusion: There are 19373 events and PseudoJet with $p_T = 20$ cut has fewer jets for each event.

In [20]:
# check the elements of first event
print(PseudoJet_filterDM_v3_pt20[0])
print('-'*80)
print(PseudoJet_filterDM_v3[0][:6])

[PseudoJet(pt=144.818, eta=0.390, phi=-0.246, mass=29.794), PseudoJet(pt=32.644, eta=0.658, phi=-0.752, mass=8.309)]
--------------------------------------------------------------------------------
[PseudoJet(pt=144.818, eta=0.390, phi=-0.246, mass=29.794), PseudoJet(pt=32.644, eta=0.658, phi=-0.752, mass=8.309), PseudoJet(pt=12.051, eta=-0.031, phi=-0.040, mass=1.622), PseudoJet(pt=3.758, eta=-4.463, phi=-3.098, mass=1.978), PseudoJet(pt=2.496, eta=0.984, phi=-0.339, mass=0.646), PseudoJet(pt=2.082, eta=-2.671, phi=-2.373, mass=0.975)]


In [24]:
# test len(v3 pt20) = len(v3)
if len(PseudoJet_filterDM_v3_pt20) == len(PseudoJet_filterDM_v3):
    print('yes')
else:
    print('no')


# check the v3 pt20 is consistent with v3 when pT > 20 GeV
sum_pt, sum_eta, sum_phi, sum_m = 0, 0, 0, 0

for i in range(len(PseudoJet_filterDM_v3_pt20)):
    for j, jet in enumerate(PseudoJet_filterDM_v3_pt20[i]):
        sum_pt += jet.pt - PseudoJet_filterDM_v3[i][j].pt
        sum_eta += jet.eta - PseudoJet_filterDM_v3[i][j].eta
        sum_phi += jet.phi - PseudoJet_filterDM_v3[i][j].phi
        sum_m += jet.mass - PseudoJet_filterDM_v3[i][j].mass
print(sum_pt, sum_eta, sum_phi, sum_m)

if sum_pt == sum_eta == sum_phi == sum_m == 0:
    print("The result of v3 with pT > 20 GeV is the same as v3 result.")
else:
    print("Please check your PseudoJet data.")

yes
0.0 0.0 0.0 0.0
The result of v3 with pT > 20 GeV is the same as v3 result.


##### Conclusion 1: pTmin_pyjet excludes the $p_T < 20$ GeV events.

##### Conclusion 2: The v3 pt20 is the same as v3 when $p_T > 20$ GeV.

In [28]:
for i in range(40):
    if len(PseudoJet_filterDM_v3_pt20[i]) >= 5:
        print(i, len(PseudoJet_filterDM_v3_pt20[i]))

7 5
8 5
10 5
17 5
19 5
21 6
25 6
27 5
31 5
32 7
33 7
39 6


In [31]:
# see the constituents of 21th event
PseudoJet_filterDM_v3_pt20[21]

[PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172),
 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663),
 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942),
 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325),
 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152),
 PseudoJet(pt=27.550, eta=-0.937, phi=0.655, mass=4.183)]

In [33]:
# pick up the constituents of 21th event
for i, jet in enumerate(PseudoJet_filterDM_v3_pt20[21][:6]):
    print(i+1, jet)
    print(jet.pt, jet.eta, jet.phi, jet.mass)
    print('-'*80)

1 PseudoJet(pt=754.721, eta=-0.269, phi=-0.352, mass=73.172)
754.7210795322543 -0.26936706270912986 -0.3523884499845895 73.17201984069227
--------------------------------------------------------------------------------
2 PseudoJet(pt=104.833, eta=-1.115, phi=1.143, mass=20.663)
104.83282762072056 -1.114822563510326 1.1430395153388633 20.66261801887006
--------------------------------------------------------------------------------
3 PseudoJet(pt=85.254, eta=0.918, phi=-2.531, mass=16.942)
85.25366784541764 0.9183119855952391 -2.5309675484291567 16.94153708924539
--------------------------------------------------------------------------------
4 PseudoJet(pt=64.974, eta=-3.034, phi=1.609, mass=11.325)
64.97417129357518 -3.033888687633622 1.6086664084075037 11.325219367653418
--------------------------------------------------------------------------------
5 PseudoJet(pt=54.199, eta=0.230, phi=-2.593, mass=12.152)
54.199148701057794 0.23044175755383908 -2.5931788854839555 12.15247456867082

#### Note: These are pyjet example.
1. This is the way to select elements and variables for the PseudoJet.  
2. This is how to pick up the constituents of each jet.

In [34]:
# Printing a few things here as a demonstration of the basic functionality
print("{0: <5} {1: >10} {2: >10} {3: >10} {4: >10} {5: >10}".format(
    "jet#", "pT", "eta", "phi", "mass", "#constit."))
for i, jet in enumerate(PseudoJet_filterDM_v3_pt20[21][:6]):
    print("{0: <5} {1: 10.3f} {2: 10.3f} {3: 10.3f} {4: 10.3f} {5: 10}".format(
        i + 1, jet.pt, jet.eta, jet.phi, jet.mass, len(jet)))

jet#          pT        eta        phi       mass  #constit.
1        754.721     -0.269     -0.352     73.172         87
2        104.833     -1.115      1.143     20.663         26
3         85.254      0.918     -2.531     16.942         33
4         64.974     -3.034      1.609     11.325         21
5         54.199      0.230     -2.593     12.152         23
6         27.550     -0.937      0.655      4.183          7


In [35]:
print("\nThe 6th jet has the following constituents:")
for constit in PseudoJet_filterDM_v3_pt20[21][5]:
    print(constit)
print("\nGet the constituents as an array (pT, eta, phi, mass):")
print(PseudoJet_filterDM_v3_pt20[21][5].constituents_array())
print("\nor (E, px, py, pz):")
print(PseudoJet_filterDM_v3_pt20[21][5].constituents_array(ep=True))


The 6th jet has the following constituents:
PseudoJet(pt=0.995, eta=-1.110, phi=0.674, mass=0.140)
PseudoJet(pt=6.002, eta=-1.017, phi=0.548, mass=0.940)
PseudoJet(pt=0.987, eta=-0.777, phi=0.579, mass=0.140)
PseudoJet(pt=1.620, eta=-0.942, phi=0.557, mass=0.000)
PseudoJet(pt=4.142, eta=-0.939, phi=0.609, mass=-0.000)
PseudoJet(pt=7.853, eta=-0.895, phi=0.741, mass=0.140)
PseudoJet(pt=6.043, eta=-0.888, phi=0.718, mass=0.938)

Get the constituents as an array (pT, eta, phi, mass):
[(0.99460959, -1.1102134 , 0.67404842,  1.39569998e-01)
 (6.0019908 , -1.01715386, 0.54768401,  9.39570010e-01)
 (0.98741752, -0.776838  , 0.57899618,  1.39569998e-01)
 (1.61963058, -0.94204098, 0.55713117,  0.00000000e+00)
 (4.14160728, -0.93889606, 0.60915875, -8.42936970e-08)
 (7.85252857, -0.89518613, 0.74074847,  1.39569998e-01)
 (6.04252958, -0.88757044, 0.71782219,  9.38269973e-01)]

or (E, px, py, pz):
[( 1.67899042, 0.7770897 , 0.62078969, -1.34546676)
 ( 9.43084008, 5.12409623, 3.12530499, -7.21346